In [14]:
import pandas as pd
from pathlib import Path

from fuzzywuzzy import fuzz, process

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
path = Path.cwd().parent.parent / 'data' / 'raw' / 'journeys.csv'
df = pd.read_csv(path)
df.shape
df.head()

(10127, 5)

,question_id,reply_id,created_at,question,reply
0,31665,18475753,2020-11-27 18:34:35,Tell us...what brand(s) did you purchase?,Uggs
1,31665,21873401,2021-02-05 17:24:45,Tell us...what brand(s) did you purchase?,Adidas
2,31665,21871152,2021-02-05 16:43:52,Tell us...what brand(s) did you purchase?,Timberland
3,31665,21873596,2021-02-05 17:28:04,Tell us...what brand(s) did you purchase?,Converse
4,31665,21871473,2021-02-05 16:52:05,Tell us...what brand(s) did you purchase?,Red tims


In [3]:
df.question_id.unique()

array([31665, 31875, 37765])

In [4]:
df_31665 = df.query('question_id==31665')
df_31665.shape

(8497, 5)

In [20]:
num_uniques = len(df_31665.reply.unique())
print(f'The nunmber of unique replies is {num_uniques}')

df_31665.reply.value_counts()

The nunmber of unique replies is 1970


Vans                                          1195
Vans                                           489
Converse                                       389
Converse                                       309
Crocs                                          218
                                              ... 
Chacon                                           1
vans, hey dudes, journeys                        1
Converse \nLoretta                               1
Vans and journey socks and cleaning spray        1
Van's, Tommy Hilfiger, Levi                      1
Name: reply, Length: 1969, dtype: int64

In [13]:
Str1 = "The supreme court case of Nixon vs The United States"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)
print(Token_Set_Ratio)

57
77
58
95


In [19]:
strings_to_match = df_31665.reply.values
strings_to_match

array(['Uggs', 'Adidas ', 'Timberland', ..., '5',
       'Adidas and Berkenstock', 'Doc Martens'], dtype=object)

In [26]:
patterns = [
    'vans',
    'converse',
    'doc marten',
    'uggs',
    'adidas',
    'crocs',
    'timberlands',
    'journey\'s',
    'dude',
    'new balance',
    'fila',
    'puma',
    'sperry',
    'birkenstock',
    'steve madden',
    'north face',
    'toms',
    'dc',
    'reebok',
    'roxy',
    'champion',
    'jan sport',
    'heelys',
    'polo',
    'nike',
    'kahken',
    'teva',
    'sorel',
    'tommy hilfiger',
    'herschel',
    'lacoste',
    'perry',
    'clarks',
    'speedy',
    'rainbow',
    'ralph lauren',
    'reefs',
    'levi\'s',
    'chaco',
    'madden girl'    
]

In [37]:
patterns

['vans',
 'converse',
 'doc marten',
 'uggs',
 'adidas',
 'crocs',
 'timberlands',
 "journey's",
 'dude',
 'new balance',
 'fila',
 'puma',
 'sperry',
 'birkenstock',
 'steve madden',
 'north face',
 'toms',
 'dc',
 'reebok',
 'roxy',
 'champion',
 'jan sport',
 'heelys',
 'polo',
 'nike',
 'kahken',
 'teva',
 'sorel',
 'tommy hilfiger',
 'herschel',
 'lacoste',
 'perry',
 'clarks',
 'speedy',
 'rainbow',
 'ralph lauren',
 'reefs',
 "levi's",
 'chaco',
 'madden girl']

In [45]:
threshold = 90
matched = [process.extractOne(str(i), patterns) for i in strings_to_match]
matched = [t[0] if t[1]>=threshold else 'no_match' for t in matched]
matched

['uggs',
 'adidas',
 'timberlands',
 'converse',
 'no_match',
 'vans',
 'vans',
 'vans',
 'converse',
 'adidas',
 'converse',
 'vans',
 'reebok',
 'converse',
 'vans',
 'vans',
 'doc marten',
 'perry',
 'dude',
 'converse',
 'no_match',
 'no_match',
 'adidas',
 'no_match',
 'adidas',
 'adidas',
 'converse',
 'vans',
 'converse',
 "levi's",
 'converse',
 'converse',
 'no_match',
 'adidas',
 'vans',
 'vans',
 'vans',
 'no_match',
 'uggs',
 'no_match',
 'uggs',
 'timberlands',
 'reebok',
 'vans',
 'no_match',
 'timberlands',
 'nike',
 'nike',
 'vans',
 'no_match',
 'timberlands',
 'no_match',
 'no_match',
 'no_match',
 'adidas',
 'new balance',
 'no_match',
 'vans',
 'vans',
 'no_match',
 'no_match',
 'rainbow',
 'no_match',
 'no_match',
 'no_match',
 'no_match',
 'no_match',
 'no_match',
 'no_match',
 'vans',
 'ralph lauren',
 'vans',
 'sperry',
 'sperry',
 'sperry',
 'sperry',
 'vans',
 'jan sport',
 'jan sport',
 'jan sport',
 'jan sport',
 'jan sport',
 'jan sport',
 'jan sport',
 'ja

In [16]:
str2Match = "apple inc"
strOptions = ["Apple Inc.","apple park","apple incorporated","iphone"]

process.extract(str2Match,strOptions)

[('Apple Inc.', 100),
 ('apple incorporated', 90),
 ('apple park', 67),
 ('iphone', 30)]

In [17]:
process.extractOne(str2Match,strOptions)

('Apple Inc.', 100)